In [66]:
import ast


import pandas as pd
import numpy as np
import math

In [67]:
def read_file(file_name):
    mode='r'
    read_ops = ['dataset_read', 'blob_get', ] #'H5VLdataset_get'
    
    write_ops = ['dataset_write', 'blob_put']
    # 'file_open' # not used, dset has file info
    other_ops = ['object_open', 'dataset_open', 'dataset_create', 'object_close', 'dataset_close' ] 

    index = 0

    records = {}
    with open(file_name, mode) as f:
        for line in f:
            try:
                rec = ast.literal_eval(line)
                # print(data)
                    
                if any(op in rec['func_name'] for op in read_ops):
                    records[index] = rec
                    records[index]['operation'] = 'read'
                    index+=1
                elif any(op in rec['func_name'] for op in write_ops):
                    records[index] = rec
                    records[index]['operation'] = 'write'
                    index+=1
                elif any(op in rec['func_name'] for op in other_ops):
                    records[index] = rec
                    op_type = rec['func_name'].split('_')[1]
                    records[index]['operation'] = op_type
                    index+=1
                    
            except:
                print("Erro line:")
                print(line)
                break
    return records

def rec_to_df(records):
    df = pd.DataFrame.from_dict(records,orient='index')
    df.replace('/mnt/ssd/mtang11/','',regex=True, inplace=True)
    df.replace('molecular_dynamics_runs/stage0000/','',regex=True, inplace=True)
    
    # df['hash_id']= df['hash_id'].astype(str)
    # df['hash_id'] = map(lambda x: x.encode('base64','strict'), df['hash_id'])
    return df

def df_to_csc(df,file_name):
    df.hash_id=df.hash_id.astype('category').cat.codes
    out_csv=file_name.replace('.log','.csv')
    # print(df.columns)
    df.to_csv(out_csv)

def get_hash_id_list(df):
    hash_id_list = list(df.hash_id.unique())
    if -1 in hash_id_list:
        hash_id_list.remove(-1) # -1 is not valid hash_id
    return hash_id_list

def hash_id_groups(df,file_name):
    # observe hash_id and blob_access_index relations
    df['access_idx'] = df.index
    df = df.fillna(0)
    df['total_read_bytes'] = df['total_read_bytes'].astype(int)
    df = df[df['hash_id'] != -1] # remove hash_id = -1 --> not hashed    
    df = df[df['func_name'].str.contains("blob")] # select only blob operations
    
    grouped = df.groupby('hash_id', group_keys=True)

    out_txt=file_name.replace('.log','.txt')
    
    with open(out_txt, 'w') as f:
        for name, group in grouped:
            # tmp_df = grouped.get_group(idx)
            # tmp_df= tmp_df.sort_values(by=['blob_access_idx'],ascending=True)
            f.write(f'hash_id = {name} \n')
            rec = group.to_dict(orient='records')
            for r in rec:
                f.write(f'{r}\n')
            f.write('\n')

In [68]:
fsim='tmp_outputs/prov-sim.log'
fagg='tmp_outputs/prov-agg.log'


sim_rec = read_file(fsim)
agg_rec = read_file(fagg)
# print(records[0])
# print(records[0]['dimensions'][0])
# print(list(blob_records.items())[0])
# print(blob_records[list(blob_records.keys())[0]])
dfsim = rec_to_df(sim_rec)
dfagg = rec_to_df(agg_rec)

df_to_csc(dfsim,fsim)
df_to_csc(dfagg,fagg)



hash_id_groups(dfsim,fsim)
hash_id_groups(dfagg,fagg)
# print(df.head(2))



AttributeError: 'DataFrame' object has no attribute 'hash_id'

In [ ]:
print(dfsim['blob_access_idx'].max())
print(dfagg['blob_access_idx'].max())

1000
2000


In [52]:
sim_hash_id = get_hash_id_list(dfsim)
agg_hash_id = get_hash_id_list(dfagg)

sim_hash_id = sim_hash_id.sort()
agg_hash_id = agg_hash_id.sort()

if sim_hash_id == agg_hash_id:
    print("Same")
else:
    print(f'agg_hash_id = {agg_hash_id}')
    print(f'agg_hash_id = {agg_hash_id}')

Same
